In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_deepfake_detector_resnet50(input_shape=(128, 128, 3), learning_rate=0.0001):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def build_deepfake_detector_xception(input_shape=(128, 128, 3), learning_rate=0.0001):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

if __name__ == '__main__':
    DATA_DIR = "data_pre"
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    CHANNELS = 3
    BATCH_SIZE = 32
    EPOCHS = 10

    # Load datasets
    train_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/train",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/val",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    choice = input("Choose model (1 for Xception, 2 for ResNet50): ")
    if choice =='1':
        # Preprocess images for Xception
        preprocess = tf.keras.applications.xception.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )
    elif choice == '2':
        # Preprocess images for ResNet50
        preprocess = tf.keras.applications.resnet50.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))

        # Build and train model
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )   


2025-06-02 22:15:44.168329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748877344.183504  106544 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748877344.188103  106544 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748877344.201261  106544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748877344.201275  106544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748877344.201278  106544 computation_placer.cc:177] computation placer alr

Found 22896 files belonging to 2 classes.


I0000 00:00:1748877347.490630  106544 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9814 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6


Found 2514 files belonging to 2 classes.


Model: "functional"

...

715/716 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5113 - loss: 0.7181

2025-06-02 22:16:19.572024: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5254', 116 bytes spill stores, 116 bytes spill loads

2025-06-02 22:16:19.771684: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5254', 4 bytes spill stores, 4 bytes spill loads

2025-06-02 22:16:20.066788: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5254', 992 bytes spill stores, 992 bytes spill loads

2025-06-02 22:16:20.070136: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5254', 3940 bytes spill stores, 3920 bytes spill loads



716/716 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5113 - loss: 0.7180

2025-06-02 22:16:27.650125: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1687', 4 bytes spill stores, 4 bytes spill loads

2025-06-02 22:16:27.941909: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1687', 220 bytes spill stores, 220 bytes spill loads

2025-06-02 22:16:28.082944: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1687', 8 bytes spill stores, 8 bytes spill loads

2025-06-02 22:16:28.331155: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1687', 4384 bytes spill stores, 4228 bytes spill loads

2025-06-02 22:16:28.387353: I external

716/716 ━━━━━━━━━━━━━━━━━━━━ 37s 38ms/step - accuracy: 0.5113 - loss: 0.7180 - val_accuracy: 0.5267 - val_loss: 0.6916
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - accuracy: 0.5216 - loss: 0.6963 - val_accuracy: 0.5366 - val_loss: 0.6887
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.5331 - loss: 0.6899 - val_accuracy: 0.5362 - val_loss: 0.6893
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.5444 - loss: 0.6875 - val_accuracy: 0.5378 - val_loss: 0.6892
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.5458 - loss: 0.6870 - val_accuracy: 0.5326 - val_loss: 0.6897
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - accuracy: 0.5462 - loss: 0.6857 - val_accuracy: 0.5497 - val_loss: 0.6877
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.5446 - loss: 0.6853 - val_accuracy: 0.5553 - val_loss: 0.6867
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.5537 - loss: 0.6843 - val_accurac

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_deepfake_detector_resnet50(input_shape=(128, 128, 3), learning_rate=0.0001):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def build_deepfake_detector_xception(input_shape=(128, 128, 3), learning_rate=0.0001):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

if __name__ == '__main__':
    DATA_DIR = "data_pre"
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    CHANNELS = 3
    BATCH_SIZE = 32
    EPOCHS = 10

    # Load datasets
    train_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/train",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/val",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    choice = input("Choose model (1 for Xception, 2 for ResNet50): ")
    if choice =='1':
        # Preprocess images for Xception
        preprocess = tf.keras.applications.xception.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )
    elif choice == '2':
        # Preprocess images for ResNet50
        preprocess = tf.keras.applications.resnet50.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))

        # Build and train model
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )   


Found 22896 files belonging to 2 classes.
Found 2514 files belonging to 2 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.5350 - loss: 0.7819 - val_accuracy: 0.5091 - val_loss: 0.7135
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.6120 - loss: 0.6554 - val_accuracy: 0.5072 - val_loss: 0.7267
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.6362 - loss: 0.6315 - val_accuracy: 0.4948 - val_loss: 0.7505
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.6645 - loss: 0.6051 - val_accuracy: 0.5099 - val_loss: 0.7683
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 18s 24ms/step - accuracy: 0.6848 - loss: 0.5865 - val_accuracy: 0.4881 - val_loss: 0.8091
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.7067 - loss: 0.5609 - val_accuracy: 0.4642 - val_loss: 0.8366
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.7267 - loss: 0.5410 - val_accuracy: 0.4626 - val_loss: 0.8555
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.7404 - loss: 0.5213 - 